In [37]:
from create_vocab import create_vocab_set
from create_sentences import create_s_dictionary

file_name = '/Users/alissavalentine/Charney rotation/project code/input/train_sentences.txt'
train_file = open(file_name)
sentences = create_s_dictionary(train_file)
train_file = open(file_name)
vocab = create_vocab_set(train_file)

In [ ]:
from create_weights import create_weights
weights = create_weights(vocab, sentences)

In [ ]:
file_name = '/Users/alissavalentine/Charney rotation/project code/input/train_sentences_copy.txt'
sample_note = []
with open(file_name) as file:
    for line in file:
        line = str(line).rstrip('\n')
        sample_note.append(line)

print(sample_note)

In [248]:
import random
import numpy

seed1 = numpy.random.RandomState(40)
seed2 = random.Random(40)

def replace_words(note, percentage, vocab, weight):
    metadata = [] # "Note #, Word #, Sentence #, Old Word, New Word"
    new_note = []
    sentence_num = 0

    ww = []
    for value in weight.values():
        ww.append(value)

    for line in note:
        line = line.rstrip('\n').rsplit(",")
        words = line[2].rsplit(" ")
        patient_id = line[0]
        challenge = line[1]

        indexed_words = []
        for word in words:
            index = words.index(word)
            char_before = words[:index]
            char_before = sum(len(i) for i in char_before) + len(char_before)
            indexed_word = word + "_" + str(index) + "_" + str(char_before)
            indexed_words.append(indexed_word)

        num_2replace = round((percentage * 0.01) * len(words))

        words_2replace = seed1.choice(indexed_words, size=num_2replace, replace=False)
        replacement_words = seed2.choices(vocab, k=num_2replace, weights=ww)

        for count, value in enumerate(words_2replace):
            old_word = value.rsplit("_")[0]
            index = int(value.rsplit("_")[1])
            char_before = value.rsplit("_")[2]
            new_word = replacement_words[count]
            words[index] = new_word
            char_after_old = int(char_before) + len(old_word) + 1
            old_word_char = str(char_before) + ":" + str(char_after_old)
            char_after_new = int(char_before) + len(new_word) + 1
            new_word_char = str(char_before) + ":" + str(char_after_new)
            word_metadata = str(patient_id) + "," + str(index) + "," + str(old_word_char) + "," + str(new_word_char) + "," + str(sentence_num) + "," + str(old_word)+ "," + str(new_word)
            metadata.append(word_metadata)

        words = " ".join(words)
        new_line = str(patient_id + ',' + challenge + ',' + words)
        new_note.append(new_line)

        sentence_num += 1

    return new_note, metadata

new_note, metadata = replace_words(sample_note, 50, list(vocab), weights)

In [ ]:
new_note, metadata = replace_words(sample_note, 50, list(vocab), weights)

In [250]:
def add_sentences(note, add_num, s_dictionary):
    meta_data = [] # "Note #, OG Sentence Count, New Sentence Count, OG Note # of New Sentence, Index of New Sentence in OG Note"
    og_sentence_count = len(note)
    new_sentence_count = og_sentence_count

    line = note[0].rstrip('\n').rsplit(",")
    patient_id = line[0]
    challenge = line[1]

    keys = seed2.choices(list(s_dictionary.keys()), k=add_num)
    for key in keys:
        sentence = seed2.choices(list(s_dictionary[key]), k=1)
        note.append(patient_id + "," + challenge + "," + sentence[0])
        og_note_id = key.rsplit(",")[0]
        new_sentence_count += 1
        index = list(s_dictionary[key]).index(sentence[0])
        meta_data.append(str(patient_id) + "," + str(og_sentence_count) + "," + str(new_sentence_count) + "," + str(og_note_id) + "," + str(index))
    return note, meta_data

In [ ]:
newnote, meta_data = add_sentences(sample_note, 3, sentences)

In [231]:
def repeat_notes(file_input, num_S2add, percent_W2replace):
    with file_input as file:
        lines = filter(None, (line.rstrip() for line in file))
        all_new_notes = []
        current_note = []
        last_id = ''
        word_metadata = []
        sentence_metadata = []
        for line in lines:
            line_sep = str(line).rstrip('\n').rsplit(",")
            current_id = line_sep[0]
            new_id = current_id + "_1"
            new_line = new_id + "," + line_sep[1] + "," + line_sep[2]
            if current_id != last_id:
                if len(current_note) > 0:
                    #print(current_note[-4:])
                    note_newwords, md_words = replace_words(current_note, percent_W2replace, list(vocab), weights)
                    word_metadata.append(md_words)
                    #print(note_newwords[-4:])
                    note_newsents, md_sent = add_sentences(note_newwords, num_S2add, sentences)
                    sentence_metadata.append(md_sent)
                    #print(note_newsents[-4:])
                    all_new_notes.append(note_newsents)
                last_id = current_id
                current_note = [new_line]
            else:
                current_note.append(new_line)

        note_newwords, md_words = replace_words(current_note, percent_W2replace, list(vocab), weights)
        word_metadata.append(md_words)
        note_newsents, md_sent = add_sentences(note_newwords, num_S2add, sentences)
        sentence_metadata.append(md_sent)
        all_new_notes.append(note_newsents)

    return all_new_notes, word_metadata, sentence_metadata

In [253]:
file_name = '/Users/alissavalentine/Charney rotation/project code/input/train_sentences_copy_long.txt'
train_file_2 = open(file_name)
new_notes, md_word, md_sentence = repeat_notes(train_file_2, 3, 50)

In [ ]:
print(md_sentence)
print(md_word)

In [202]:
import os
os.chdir("/Users/alissavalentine/Charney rotation/project code/output")


In [203]:
synthetic_file = open("synthetic_notes.txt", "w")
with synthetic_file as file:
    for note in new_notes:
        for line in note:
            file.writelines(line)
            file.writelines("\n")
synthetic_file.close()

In [204]:
word_md_file = open("word_metadata.txt", "w")
with word_md_file as file:
    file.writelines("Note #, Word #, Sentence #, Old Word, New Word\n")
    for note_changes in md_word:
        for line in note_changes:
            file.writelines(str(line))
            file.writelines("\n")
word_md_file.close()

In [205]:
sentence_md_file = open("sentence_metadata.txt", "w")
with sentence_md_file as file:
    file.writelines("Note #, OG Sentence Count, New Sentence Count, OG Note # of New Sentence, Index of New Sentence in OG Note\n")
    for note_changes in md_sentence:
        for line in note_changes:
            file.writelines(str(line))
            file.writelines("\n")
sentence_md_file.close()